In [12]:
from dotenv import load_dotenv
from glob import glob
from pprint import pprint
import os

In [13]:
load_dotenv()

True

In [14]:
# 문서 로딩
text_files = glob(os.path.join('data', '*_KR.txt'))
print(text_files)

['data\\samsung_KR.txt', 'data\\skHynix_KR.txt', 'data\\리비안_KR.txt', 'data\\테슬라_KR.txt']


In [15]:
# 문서 로더
from langchain_community.document_loaders import TextLoader

loader = TextLoader(text_files[0], encoding='utf-8')
data = loader.load()
len(data)

1

In [16]:
from tqdm import tqdm

data  = []

for text_file in tqdm(text_files):
    loader = TextLoader(text_file, encoding='utf-8')
    data += loader.load()
    # data.append(loader.load())

len(data)


100%|██████████| 4/4 [00:00<00:00, 2005.40it/s]


4

In [17]:
# 텍스트 분할기(Text Splitters)
char_count = [len(doc.page_content) for doc in data]
print(char_count)

[743, 674, 514, 451]


In [18]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=250, # 청크의 크기
    chunk_overlap=50, # 청크 중 중복되는 크기
    separator='\n\n'
)

texts = text_splitter.split_documents(data)
print(f"생성된 텍스크 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")

Created a chunk of size 282, which is longer than the specified 250
Created a chunk of size 268, which is longer than the specified 250


생성된 텍스크 청크 수: 13
각 청크의 길이: [164, 203, 217, 152, 149, 226, 123, 170, 175, 282, 52, 268, 180]


In [19]:
# 임베딩 모델(Embeddings)
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model='text-embedding-3-small',
)

In [20]:
# 벡터 저장소(Vector Stores)
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    collection_name='chroma_test',
    persist_directory='./chroma_db'
)

print(f'벡터 저장소에 저장된 문서: {vectorstore._collection.count()}')

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


벡터 저장소에 저장된 문서: 26


In [21]:
# RAG 프롬프트 템플릿 파이프라인 구성
from langchain.prompts import ChatPromptTemplate

template = '''다음 맥락만을 바탕으로 질문에 답하세요.
외부 정보나 지식을 사용하지 마세요.
답변이 문맥에 맞지 않는 경우 답변 '잘 모르겠습니다.'

[Context]
{context}

[Question]
{question}

[Answer]
'''

prompt = ChatPromptTemplate.from_template(template)

prompt.pretty_print()

================================ Human Message =================================

다음 맥락만을 바탕으로 질문에 답하세요.
외부 정보나 지식을 사용하지 마세요.
답변이 문맥에 맞지 않는 경우 답변 '잘 모르겠습니다.'

[Context]
{context}

[Question]
{question}

[Answer]



In [22]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0, max_tokens=100)

In [23]:
# 문서 검색기 (retriever)
retriever = vectorstore.as_retriever(
    search_kwargs={'k':2}
)

In [ ]:
# 문서 포맷터 함수
def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

retriever_chain = retriever | format_docs

response = retriever_chain.invoke('삼성전자 파운드리 사업 시장 점유율은 어떤가요?')

pprint(response)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


('삼성전자 파운드리 사업의 시장 점유율이 지속적으로 하락하고 있습니다. 2025년 1분기 기준, 삼성전자의 파운드리 시장 점유율은 '
 '17.7%로, 전년 동기 대비 감소했습니다. 반면, TSMC는 시장 점유율을 확대하며 67.6%를 기록하여 격차가 더욱 커지고 있는 '
 '상황입니다.\n'
 '\n'
 '삼성전자 파운드리 사업의 시장 점유율이 지속적으로 하락하고 있습니다. 2025년 1분기 기준, 삼성전자의 파운드리 시장 점유율은 '
 '17.7%로, 전년 동기 대비 감소했습니다. 반면, TSMC는 시장 점유율을 확대하며 67.6%를 기록하여 격차가 더욱 커지고 있는 '
 '상황입니다.')


In [25]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {'context': retriever_chain, 'question':RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = '삼성전자 파운드리 사업 시장 점유율은 어떤가요?'
response = rag_chain.invoke(query)

In [26]:
response

'삼성전자 파운드리 사업의 시장 점유율은 2025년 1분기 기준으로 17.7%입니다.'